In [8]:
import os
from dotenv import load_dotenv
import pandas as pd
from data_loader import DataLoader
from data_cleaner import DataCleaner
from feature_engineer import FeatureEngineer
from rfm_calculator import RFMCalculator
from saver import ModelSaver
from churn import ChurnModel

def main():
    load_dotenv()

    
    best_path = os.getenv('BEST_PATH')
    cust_path = os.getenv('CUST_PATH')
    trx_path = os.getenv('TRX_PATH')

    if not all([best_path, cust_path, trx_path]):
        raise ValueError("Path yok")

    
    loader = DataLoader(best_path, cust_path, trx_path)
    best, cust, trx = loader.load_data()

    
    merged_trx = pd.merge(trx, cust, on='cb_customer_id', how='inner')
    df = pd.merge(merged_trx, best, on='unique_customer_id', how='left')
    df.drop_duplicates(inplace=True)

    
    cleaner = DataCleaner(df)
    df = cleaner.clean_gender_data()
    df = cleaner.convert_date_columns()
    df = cleaner.drop_unnecessary_columns()

    
    engineer = FeatureEngineer(df)
    df = engineer.add_feature_columns()

    
    rfm_calculator = RFMCalculator(df)
    rfm = rfm_calculator.calculate_rfm()
    rfm = rfm_calculator.assign_rfm_scores()

    
    processed_data = engineer.add_additional_features(rfm)

    
    churn_model = ChurnModel(processed_data)
    churn_model.run()

   
    saver = ModelSaver()
    saver.save_model(churn_model, 'churn_model')

if __name__ == "__main__":
    main()

Test Seti Doğruluk Skoru: 0.7054292692090927

Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       0.75      0.77      0.76     16415
           1       0.62      0.59      0.61     10200

    accuracy                           0.71     26615
   macro avg       0.69      0.68      0.69     26615
weighted avg       0.70      0.71      0.70     26615

Model saved to models\churn_model.pkl
